In [46]:
import ast
import json
import pandas as pd
import numpy as np

In [47]:
#read the result data you want
df = pd.read_csv('result.txt')

In [48]:
with open('result.txt') as f:
    data = f.readlines()
    data = [item.replace("'", '"') for item in data]
    res = json.dumps(data)

In [49]:
table = pd.DataFrame()
result = json.loads(res)
for a in result:
        add = ast.literal_eval(a)
        print(add,type(add))
        table = table.append(add, ignore_index=True)

{'Dataset': 'cora', 'model': 'featwalk', 'f1_micro': 0.8478633936062095, 'f1_macro': 0.8333061806893458, 'roc_score': 0.990658159572821, 'ap_score': 0.9824548166717377, 'best': {'alpha1': 0.705336059137853, 'evaluation': 0, 'num_paths': 48.0, 'path_length': 25.0, 'win_size': 12.0}} <class 'dict'>
{'Dataset': 'cora', 'model': 'netmf', 'f1_micro': 0.8452824140071347, 'f1_macro': 0.8366226540364392, 'roc_score': 0.9902278840164334, 'ap_score': 0.9926354714233097, 'best': {'window': 9.0}} <class 'dict'>
{'Dataset': 'cora', 'model': 'deepwalk', 'f1_micro': 0.8013382351938121, 'f1_macro': 0.7908166083319863, 'roc_score': 0.9265344994581048, 'ap_score': 0.8794987961721727, 'best': {'number_walks': 53.0, 'walk_length': 41.0, 'window_size': 44.0}} <class 'dict'>
{'Dataset': 'cora', 'model': 'node2vec', 'f1_micro': 0.8589396430008662, 'f1_macro': 0.8502424881609688, 'roc_score': 0.9218770816155317, 'ap_score': 0.9141549148302945, 'best': {'length': 32.0, 'p': 0.4233821836207442, 'q': 1.818351414

In [50]:
#test loop
test = ast.literal_eval(result[1])

In [51]:
# #link prediction drop
# table = table.drop(columns=['best','f1_macro','f1_micro'])

In [52]:
#node_classification drop
table = table.drop(columns=['best','ap_score','roc_score'])

In [53]:
table

,Dataset,f1_macro,f1_micro,model
0,cora,0.833306,0.847863,featwalk
1,cora,0.836623,0.845282,netmf
2,cora,0.790817,0.801338,deepwalk
3,cora,0.850242,0.858940,node2vec
4,cora,0.853608,0.870020,dgi
5,cora,0.789942,0.803175,gae
6,cora,0.871218,0.879622,can_new
7,cora,0.146686,0.340842,can_original
8,flickr,0.789068,0.790985,featwalk
9,flickr,0.593379,0.597038,netmf


In [54]:
# #generate link prediction
# t1 = table.drop(columns=['roc_score','ap_score'])
# t1['  '] = table['ap_score']
# t1[' '] = 'ap_score'
# t2 = table.drop(columns=['ap_score','roc_score'])
# t2['  '] = table['roc_score']
# t2[' '] = 'roc_score'

In [55]:
#generate node classification
t1 = table.drop(columns=['f1_macro','f1_micro'])
t1['  '] = table['f1_macro']
t1[' '] = 'f1_macro'
t2 = table.drop(columns=['f1_macro','f1_micro'])
t2['  '] = table['f1_micro']
t2[' '] = 'f1_micro'

In [56]:
pf = t1.append(t2)
pf['modeltype'] =0

In [57]:
#assign deep or shallow
col_is_shallow = (pf['model'].isin(['featwalk','netmf','deepwalk','node2vec']))
col_is_not_shallow = (~pf['model'].isin(['featwalk','netmf','deepwalk','node2vec']))

pf.loc[col_is_shallow, 'modeltype'] = 'shallow'
pf.loc[col_is_not_shallow,'modeltype'] = 'deep'

In [58]:
pf = pf.pivot(index=['modeltype','model'],values=['  '],columns=['Dataset',' '])

In [59]:
pf = pf.reindex(sorted(pf.columns), axis=1)

In [60]:
pf

Dataset                     cora              flickr          
                        f1_macro  f1_micro  f1_macro  f1_micro
modeltype model                                               
deep      can_new       0.871218  0.879622       NaN       NaN
          can_original  0.146686  0.340842       NaN       NaN
          dgi           0.853608  0.870020  0.173771  0.221971
          gae           0.789942  0.803175  0.021943  0.109597
shallow   deepwalk      0.790817  0.801338  0.590223  0.595056
          featwalk      0.833306  0.847863  0.789068  0.790985
          netmf         0.836623  0.845282  0.593379  0.597038
          node2vec      0.850242  0.858940  0.566749  0.570732

In [61]:
pf.to_csv('out.csv',index=False)